In [1]:
from llama_index.core.graph_stores import SimpleGraphStore


In [3]:
from llama_index.core import SimpleDirectoryReader, Settings
documents = SimpleDirectoryReader(input_files = ['Wollastonite.md']).load_data()
print(len(documents))
print(f"Document Metadata: {documents[0].metadata}")

9
Document Metadata: {'file_path': 'Wollastonite.md', 'file_name': 'Wollastonite.md', 'file_size': 2651, 'creation_date': '2024-10-03', 'last_modified_date': '2024-10-03'}


In [5]:
# --->: Set up the local embedding model and LLM
# Set embedding model
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name='nomic-embed-text',
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

llm = Ollama(model='mistral', request_timeout=1000.0)
Settings.embed_model = ollama_embedding
Settings.llm = llm


In [6]:
graph_store = SimpleGraphStore()

In [7]:
from llama_index.core import KnowledgeGraphIndex
# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=2,
    show_progress=True,
)

In [8]:
query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)

In [24]:
retriever = index.as_retriever(similarity_top_k=2)

In [28]:
nodes = retriever.retrieve("What is Wollastonite?")
nodes[2].node.get_text()

"#Calcium_additive #raise_ph #Wollastonite #Silicon_additive #buffer_pH #Calcium\nGrowers  turn to Wollastonite for:\n- Its **liming** capability.  Wollastonite's dissolution rate is slower than agricultural lime, offering a buffering effect against rapid pH changes. This makes Wollastonite beneficial in areas with fluctuating acidity levels.\n- Adding **Silicon**.\n- Adding **Calcium**.\nWollastonite's pH buffering effect and Silicon content contribute to pest control and powdery mildew suppression, although the exact mechanisms are not fully understood."

In [22]:
from IPython.display import Markdown, display
response = query_engine.query(
    "Does Wollastonite provide the soil with Calcium?",
)
display(Markdown(f"<b>{response}</b>"))

<b> The provided information does not indicate that Wollastonite provides calcium to the soil directly. However, it is formed by the presence of limestone, which contains calcium carbonate and can release calcium into the soil when it weathers.</b>

In [17]:
from pyvis.network import Network
import io
g = index.get_networkx_graph()
print(f"Number of nodes: {g.number_of_nodes()}")
print(f"Number of edges: {g.number_of_edges()}")
net = Network(notebook=True,cdn_resources="in_line",directed=True)
net.from_nx(g)
# Instead of net.show('kg.html'), use this:
html = net.generate_html()
with io.open('kg.html', 'w', encoding='utf-8') as f:
    f.write(html)

# If you're in a Jupyter notebook and want to display the graph:
from IPython.display import IFrame
IFrame(src='kg.html', width='100%', height='500px')

Number of nodes: 41
Number of edges: 33


In [20]:
from pyvis.network import Network
import io

g = index.get_networkx_graph()
print(f"Number of nodes: {g.number_of_nodes()}")
print(f"Number of edges: {g.number_of_edges()}")

net = Network(notebook=True, cdn_resources="in_line", directed=True, height="500px", width="100%")
net.from_nx(g)

# Add some styling
net.set_edge_smooth('dynamic')
net.toggle_physics(True)
net.show_buttons(filter_=['physics'])

# Adjust node size and color
for node in net.nodes:
    node['size'] = 20
    node['color'] = '#00ff1e'  # Green color

# Adjust edge width
for edge in net.edges:
    edge['width'] = 2

# Generate and save HTML
html = net.generate_html()
with io.open('kg.html', 'w', encoding='utf-8') as f:
    f.write(html)

# If you're in a Jupyter notebook and want to display the graph:
from IPython.display import IFrame
IFrame(src='kg.html', width='100%', height='500px')

Number of nodes: 41
Number of edges: 33


In [ ]:
# Sample data
text = """
Alice is a software engineer at TechCorp. She works on AI projects.
Bob is a data scientist who collaborates with Alice on machine learning models.
TechCorp is a technology company specializing in artificial intelligence.
"""

# Create a Relik pipeline
pipeline = relik.Pipeline()

# Process the text and generate a knowledge graph
knowledge_graph = pipeline.process(text)

In [ ]:
# Print the generated triples
print("Generated Knowledge Graph Triples:")
for triple in knowledge_graph.triples:
    print(triple)